In [1]:
from converter import *
romajiToJapanese('hazusu')

'はずす'

In [1]:
import re
from bs4 import BeautifulSoup
import urllib
from tqdm import tqdm
import pickle

# URL_LVL_DICT_RAD,URL_LVL_DICT_KAN,URL_LVL_DICT_VOC={},{},{}
# for i in tqdm(range(1,61)):
#     soup = BeautifulSoup(urllib.request.urlopen(f"http://wanikani.com/level/{i}") )
#     for j in soup.find_all('a', {'href': re.compile(r'radicals/')}):
#         URL_LVL_DICT_RAD[i]=URL_LVL_DICT_RAD.get(i,[])+['http://wanikani.com'+j['href']]
#     for j in soup.find_all('a', {'href': re.compile(r'kanji/')}):
#         URL_LVL_DICT_KAN[i]=URL_LVL_DICT_KAN.get(i,[])+['http://wanikani.com'+j['href']]
#     for j in soup.find_all('a', {'href': re.compile(r'vocabulary/')}):
#         URL_LVL_DICT_VOC[i]=URL_LVL_DICT_VOC.get(i,[])+['http://wanikani.com'+j['href']]

with open('urls.pkl', 'rb') as inp:
    URL_LVL_DICT_RAD,URL_LVL_DICT_KAN,URL_LVL_DICT_VOC=pickle.load(inp)

In [2]:
bad_rads={1: [8, 9, 20], 3: [8, 18], 4: [3, 13, 28], 5: [9, 18, 22], 8: [1], 9: [3, 8, 14, 16, 17, 19], 10: [3], 11: [0, 1, 12], 12: [10], 13: [4], 14: [0], 15: [0, 5], 16: [4], 20: [0], 23: [3], 24: [3], 26: [2], 30: [0], 32: [3], 33: [0], 41: [0], 42: [2], 50: [1]}
# for i in tqdm(URL_LVL_DICT_RAD.keys()):
#     for j in range(len(URL_LVL_DICT_RAD[i])):
#         try:
#             soup = BeautifulSoup(urllib.request.urlopen(URL_LVL_DICT_RAD[i][j]))
#             a=soup.find("span", class_="radical-icon").text
#             if not a:
#                 bad_rads[i]=bad_rads.get(i,[])+[j]
#                 link_pic = soup.find('img',class_="radical-image")['src']
#                 urllib.request.urlretrieve(link_pic,soup.head.title.text.split('/')[2].strip().split('/')[-1].lower()+'.png')
#         except:
#             print(i,j)
#             continue

In [2]:
def mnemonics(url,rads=False):
    if rads:
        return BeautifulSoup(urllib.request.urlopen(url,timeout = 120)).find('section', class_="mnemonic-content mnemonic-content--new").text.replace('\n\n Hints\n',' ').replace('\n',' ').strip()
    loc = BeautifulSoup(urllib.request.urlopen(url,timeout = 120)).find_all('section', class_="mnemonic-content mnemonic-content--new")
    return [loc[0].text.replace('\n\n Hints\n',' ').replace('\n',' ').strip(),loc[1].text.replace('\n\n Hints\n',' ').replace('\n',' ').strip()]
def meanings(url):
    soup = BeautifulSoup(urllib.request.urlopen(url,timeout = 120) )
    loc=list(map(lambda x: x.text.lower(),soup.find_all("p", class_="text-gray-700")))
    newm=[]
    for m in loc:
        newm+=m.split(', ')
    return newm
def hyerogliph(url):#KAN AND VOC
    soup = BeautifulSoup(urllib.request.urlopen(url,timeout = 120))
    return soup.head.title.text.split('/')[2].strip().split('/')[-1]
def readings(url): 
    soup = BeautifulSoup(urllib.request.urlopen(url,timeout = 120))
    loc=soup.find("div", class_="span4 reading--kunyomi")
    if loc:
        return [loc.text.replace('Kun’yomi','').strip().split(', '),
        soup.find("div", class_="span4 reading--onyomi muted-content").text.replace('On’yomi','').strip().split(', '),
        'kun']
    else:
        return [soup.find("div", class_="span4 reading--kunyomi muted-content").text.replace('Kun’yomi','').strip().split(', '),
        soup.find("div", class_="span4 reading--onyomi").text.replace('On’yomi','').strip().split(', '),
        'on']
def context_sents(url):
    return list(filter(None, BeautifulSoup(urllib.request.urlopen(url,timeout = 120)).find('section', id="context", class_="context-sentence").text.replace('Context','').replace(' Sentences','').split('\n')))

In [3]:
with open('DATA.pkl', 'rb') as inp:
    DATA=pickle.load(inp)

In [34]:
#endless_kan=done!
# from wanikani import *
# import time
# def endless_kan(lvl,link,wait=10):
#     try:
#         DATA[lvl-1]['kan'].append(kan(link,hyerogliph(link),meanings(link),*readings(link),*mnemonics(link),lvl))
#         return
#     except Exception as e:
#         print(str(e))
#         time.sleep(wait)
#         if wait<60:
#             wait+=1
#         endless_kan(lvl,link,wait)
    
# for lvl in range(61,61):
#     for link in tqdm(URL_LVL_DICT_KAN[lvl]):
#         endless_kan(lvl,link)
#     with open('DATA.pkl', 'wb') as outp:
#         pickle.dump(DATA, outp, pickle.HIGHEST_PROTOCOL)


 53%|█████▎    | 18/34 [04:55<04:32, 17.01s/it]

HTTPSConnectionPool(host='translate.google.com', port=443): Read timed out.


 97%|█████████▋| 33/34 [13:53<00:18, 18.68s/it]

HTTPSConnectionPool(host='translate.google.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:997)')))


100%|██████████| 34/34 [15:28<00:00, 27.32s/it]


In [15]:
#endless_voc
from wanikani import *
def endless_voc(lvl,link,wait=1):
    try:
        if len(loc:=meanings(link))>1:
            meanings_=loc[:-1]
        else:
            meanings_=loc
        reading=list(map(lambda x:x.text.strip(), BeautifulSoup(urllib.request.urlopen(link,timeout = 120)).find_all('p', class_="pronunciation-variant")))
        DATA[lvl-1]['voc'].append(voc(link,hyerogliph(link),meanings_,reading,*mnemonics(link),context_sents(link),lvl))
        return
    except Exception as e:
        print(str(e))
        time.sleep(wait)
        if wait<10:
            wait+=0.5
        endless_voc(lvl,link,wait)

for lvl in tqdm(range(56,61)):
    for i in tqdm(range(len(DATA[lvl-1]['voc']),len(URL_LVL_DICT_VOC[lvl]))):
        link=URL_LVL_DICT_VOC[lvl][i]
        endless_voc(lvl,link)
    with open('DATA.pkl', 'wb') as outp:
        pickle.dump(DATA, outp, pickle.HIGHEST_PROTOCOL)        

0it [00:00, ?it/s]/5 [00:00<?, ?it/s]
 20%|██        | 1/5 [00:00<00:00,  4.83it/s]

The read operation timed out


The read operation timed out


100%|██████████| 5/5 [2:19:53<00:00, 1678.74s/it]


In [ ]:
# from wanikani import *

# for lvl in tqdm(range(32?,61)):
    # for j in range(len(URL_LVL_DICT_RAD[lvl])):
    #     link=URL_LVL_DICT_RAD[lvl][j]
    #     meaning = BeautifulSoup(urllib.request.urlopen(link,timeout = 120)).head.title.text.split('/')[2].strip().split('/')[-1].lower()
    #     mnem=mnemonics(link,True)
    #     if (lvl in bad_rads) and (j in bad_rads[lvl]):
    #         path='rads/'+meaning+'.png'
    #         DATA[lvl-1]['rad'].append(rad(link,None,path,meaning,mnem,lvl))
    #     else:
    #         hye=BeautifulSoup(urllib.request.urlopen(link,timeout = 120)).find("span", class_="radical-icon").text.lower()
    #         DATA[lvl-1]['rad'].append(rad(link,hye,None,meaning,mnem,lvl))

    # for link in URL_LVL_DICT_KAN[lvl]:
    #     DATA[lvl-1]['kan'].append(kan(link,hyerogliph(link),meanings(link),*readings(link),*mnemonics(link),lvl))
    
    # for link in URL_LVL_DICT_VOC[lvl]:
    #     reading=list(map(lambda x:x.text.strip(), BeautifulSoup(urllib.request.urlopen(link,timeout = 120)).find_all('p', class_="pronunciation-variant")))
    #     DATA[lvl-1]['voc'].append(voc(link,hyerogliph(link),meanings(link)[:-1],reading,*mnemonics(link),context_sents(link),lvl))

In [23]:
with open('DATA.pkl', 'wb') as outp:
    pickle.dump(DATA, outp, pickle.HIGHEST_PROTOCOL)

In [2]:
#rad_hyerogliph/image
# f = urllib.request.urlopen(URL_LVL_DICT_RAD[1][8]) 
# soup = BeautifulSoup(f)
# a=soup.find("span", class_="radical-icon").text
# link_pic=''
# if not a:
#     link_pic = soup.find('img',class_="radical-image")['src']
#     name='gun.png'
#     urllib.request.urlretrieve(link_pic,name)
# print(a,link_pic)

 https://files.wanikani.com/xg8ygnendcn6on94dwbl3t0yq2w9


In [13]:
#meanings
# f = urllib.request.urlopen(URL_LVL_DICT_KAN[20][0]) 
# soup = BeautifulSoup(f)
# list(map(lambda x: x.text.lower(),soup.find_all("p", class_="text-gray-700")))

['abundant', 'plentiful']

In [16]:
#hierogliphs for KAN and VOC!
# f = urllib.request.urlopen(URL_LVL_DICT_VOC[1][1]) 
# soup = BeautifulSoup(f)
# soup.head.title.text.split('/')[2].strip().split('/')[-1]

'一人'

In [18]:
#mnem_meaning _ reading
# f = urllib.request.urlopen(URL_LVL_DICT_VOC[1][1]) 
# soup = BeautifulSoup(f)
# l=soup.find_all('section', class_="mnemonic-content mnemonic-content--new")
# print(l[0].text.replace('\n\n Hints\n',' ').replace('\n',' ').strip())
# print('-----------------------------------------------------')
# print(l[1].text.replace('\n\n Hints\n',' ').replace('\n',' ').strip())

When there's one person, what are they? Well, they're either just going to be simply one person or alone.
-----------------------------------------------------
The reading for this vocab doesn't follow any rules you learned previously. It's an exception, which means you have to learn the reading separately too. Spend a few moments trying to remember this word, look away for 10 seconds, and then try to recall its reading. Could you do it? Try again, this time in thirty seconds. Did you do it again? Okay, go ahead and move on. You may know it now, though you'll need to recall it again in the next 5-10 minutes if you want to remember it for good. Alternatively, if you know the reading for the vocab word 一つ (in the same level as this word), you can know that it uses the same reading!


In [43]:
#readings _ VOC
# f = urllib.request.urlopen(URL_LVL_DICT_VOC[31][8]) 
# soup = BeautifulSoup(f)
# print(soup.head.title.text)
# for i in soup.find_all('p', class_="pronunciation-variant"):
#     print(i.text)

WaniKani / Vocabulary / ハチの巣
はちのす
ハチのす


In [38]:
#main_readings
# f = urllib.request.urlopen(URL_LVL_DICT_KAN[30][10]) 
# soup = BeautifulSoup(f)
# print(soup.head.title.text)
# for i in soup.find_all("div", class_="span4 reading--onyomi"):
#     print(i.text.replace('On’yomi','').strip())
# for i in soup.find_all("div", class_="span4 reading--kunyomi"):
#     print(i.text.replace('Kun’yomi','').strip())

WaniKani / Kanji / 越
えつ


In [39]:
#secondary_readings
# f = urllib.request.urlopen(URL_LVL_DICT_KAN[30][10]) 
# soup = BeautifulSoup(f)
# print(soup.head.title.text)
# for i in soup.find_all("div", class_="span4 reading--kunyomi muted-content"):
#     print(i.text.replace('Kun’yomi','').strip())
# for i in soup.find_all("div", class_="span4 reading--onyomi muted-content"):
#     print(i.text.replace("On’yomi",'').strip())

WaniKani / Kanji / 越
こ


In [49]:
#contexrt sents
# f = urllib.request.urlopen(URL_LVL_DICT_VOC[1][0]) 
# soup = BeautifulSoup(f)
# print(soup.head.title.text)
# soup.find('section', id="context", class_="context-sentence").text

WaniKani / Vocabulary / 大人


'\nContext\n\n\n\nContext Sentences\n\nこれは、大人のりょうきんです。\nThis is the adult price.\n\n\n大人は三人だけです。\nThere are only three adults.\n\n\n大人たちはいざかやにいった。\nThe adults went to an izakaya.\n\n'